In [ ]:
!pip install autogluon==0.7.0


In [1]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

In [2]:

from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.common.utils.utils import setup_outputdir
from autogluon.core.utils.loaders import load_pkl
from autogluon.core.utils.savers import save_pkl
import os.path

"""
@author: Lyle
"""

class MultilabelPredictor:
    """ Tabular Predictor for predicting multiple columns in table.
        Creates multiple TabularPredictor objects which you can also use individually.
        You can access the TabularPredictor for a particular label via: `multilabel_predictor.get_predictor(label_i)`

        Parameters ---------- labels : List[str] The ith element of this list is the column (i.e. `label`) predicted
        by the ith TabularPredictor stored in this object. path : str, default = None Path to directory where models
        and intermediate outputs should be saved. If unspecified, a time-stamped folder called "AutogluonModels/ag-[
        TIMESTAMP]" will be created in the working directory to store all models. Note: To call `fit()` twice and
        save all results of each fit, you must specify different `path` locations or don't specify `path` at all.
        Otherwise files from first `fit()` will be overwritten by second `fit()`. Caution: when predicting many
        labels, this directory may grow large as it needs to store many TabularPredictors. problem_types : List[str],
        default = None The ith element is the `problem_type` for the ith TabularPredictor stored in this object.
        eval_metrics : List[str], default = None The ith element is the `eval_metric` for the ith TabularPredictor
        stored in this object. consider_labels_correlation : bool, default = True Whether the predictions of multiple
        labels should account for label correlations or predict each label independently of the others. If True,
        the ordering of `labels` may affect resulting accuracy as each label is predicted conditional on the previous
        labels appearing earlier in this list (i.e. in an auto-regressive fashion). Set to False if during inference
        you may want to individually use just the ith TabularPredictor without predicting all the other labels.
        kwargs : Arguments passed into the initialization of each TabularPredictor.

    """

    multi_predictor_file = 'multilabel_predictor.pkl'

    def __init__(self, labels, path=None, problem_types=None, eval_metrics=None, consider_labels_correlation=True,
                 **kwargs):
        self.model_root = None
        if len(labels) < 2:
            raise ValueError(
                "MultilabelPredictor is only intended for predicting MULTIPLE labels (columns), use TabularPredictor "
                "for predicting one label (column).")
        if (problem_types is not None) and (len(problem_types) != len(labels)):
            raise ValueError("If provided, `problem_types` must have same length as `labels`")
        if (eval_metrics is not None) and (len(eval_metrics) != len(labels)):
            raise ValueError("If provided, `eval_metrics` must have same length as `labels`")
        self.path = setup_outputdir(path, warn_if_exist=False)
        self.labels = labels
        self.consider_labels_correlation = consider_labels_correlation
        self.predictors = {}  # key = label, value = TabularPredictor or str path to the TabularPredictor for this label
        if eval_metrics is None:
            self.eval_metrics = {}
        else:
            self.eval_metrics = {labels[i]: eval_metrics[i] for i in range(len(labels))}
        problem_type = None
        eval_metric = None
        for i in range(len(labels)):
            label = labels[i]
            path_i = self.path + "Predictor_" + label
            if problem_types is not None:
                problem_type = problem_types[i]
            if eval_metrics is not None:
                eval_metric = eval_metrics[i]
            self.predictors[label] = TabularPredictor(label=label, problem_type=problem_type, eval_metric=eval_metric,
                                                      path=path_i, **kwargs)

    def fit(self, train_data, tuning_data=None, **kwargs):
        """ Fits a separate TabularPredictor to predict each of the labels.

            Parameters
            ----------
            train_data, tuning_data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                See documentation for `TabularPredictor.fit()`.
            kwargs :
                Arguments passed into the `fit()` call for each TabularPredictor.
        """
        if isinstance(train_data, str):
            train_data = TabularDataset(train_data)
        if tuning_data is not None and isinstance(tuning_data, str):
            tuning_data = TabularDataset(tuning_data)
        train_data_og = train_data.copy()
        if tuning_data is not None:
            tuning_data_og = tuning_data.copy()
        else:
            tuning_data_og = None
        save_metrics = len(self.eval_metrics) == 0
        for i in range(len(self.labels)):
            label = self.labels[i]
            predictor = self.get_predictor(label)
            if not self.consider_labels_correlation:
                labels_to_drop = [l for l in self.labels if l != label]
            else:
                labels_to_drop = [self.labels[j] for j in range(i + 1, len(self.labels))]
            train_data = train_data_og.drop(labels_to_drop, axis=1)
            if tuning_data is not None:
                tuning_data = tuning_data_og.drop(labels_to_drop, axis=1)
            print(f"Fitting TabularPredictor for label: {label} ...")
            predictor.fit(train_data=train_data, tuning_data=tuning_data, **kwargs)
            self.predictors[label] = predictor.path
            if save_metrics:
                self.eval_metrics[label] = predictor.eval_metric
        self.save()

    def predict(self, data, **kwargs):
        """ Returns DataFrame with label columns containing predictions for each label.

            Parameters ---------- data : str or autogluon.tabular.TabularDataset or pd.DataFrame Data to make
            predictions for. If label columns are present in this data, they will be ignored. See documentation for
            `TabularPredictor.predict()`. kwargs : Arguments passed into the predict() call for each TabularPredictor.
        """
        return self._predict(data, as_proba=False, **kwargs)

    def predict_proba(self, data, **kwargs):
        """ Returns dict where each key is a label and the corresponding value is the `predict_proba()` output for just that label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to make predictions for. See documentation for `TabularPredictor.predict()` and `TabularPredictor.predict_proba()`.
            kwargs :
                Arguments passed into the `predict_proba()` call for each TabularPredictor (also passed into a `predict()` call).
        """
        return self._predict(data, as_proba=True, **kwargs)

    def evaluate(self, data, **kwargs):
        """ Returns dict where each key is a label and the corresponding value is the `evaluate()` output for just that label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to evalate predictions of all labels for, must contain all labels as columns. See documentation for `TabularPredictor.evaluate()`.
            kwargs :
                Arguments passed into the `evaluate()` call for each TabularPredictor (also passed into the `predict()` call).
        """
        data = self._get_data(data)
        eval_dict = {}
        for label in self.labels:
            print(f"Evaluating TabularPredictor for label: {label} ...")
            predictor = self.get_predictor(label)
            eval_dict[label] = predictor.evaluate(data, **kwargs)
            if self.consider_labels_correlation:
                data[label] = predictor.predict(data, **kwargs)
        return eval_dict

    def save(self):
        """ Save MultilabelPredictor to disk. """
        for label in self.labels:
            if not isinstance(self.predictors[label], str):
                self.predictors[label] = self.predictors[label].path
        save_pkl.save(path=self.path + self.multi_predictor_file, object=self)
        print(f"MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('{self.path}')")

    @classmethod
    def load(cls, path):
        """ Load MultilabelPredictor from disk `path` previously specified when creating this MultilabelPredictor. """
        predictor_instance = load_pkl.load(path=os.path.join(path, cls.multi_predictor_file))
        predictor_instance.model_root = path
        return predictor_instance

    def get_predictor(self, label):
        """ Returns TabularPredictor which is used to predict this label. """
        predictor = self.predictors[label]
        if isinstance(predictor, str):
            path_elements = predictor.split("/")
            path_relative_to_root = path_elements[-2] + "/" + path_elements[-1]
            return TabularPredictor.load(path=os.path.join(self.model_root, path_relative_to_root))
        return predictor

    def _get_data(self, data):
        if isinstance(data, str):
            return TabularDataset(data)
        return data.copy()

    def _predict(self, data, as_proba=False, **kwargs):
        data = self._get_data(data)
        if as_proba:
            predproba_dict = {}
        for label in self.labels:
            #             print(f"Predicting with TabularPredictor for label: {label} ...")
            predictor = self.get_predictor(label)
            if as_proba:
                predproba_dict[label] = predictor.predict_proba(data, as_multiclass=True, **kwargs)
            data[label] = predictor.predict(data, **kwargs)
        if not as_proba:
            return data[self.labels]
        else:
            return predproba_dict


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# -*- coding: utf-8 -*-
"""
Created on Sun Jul 10 12:05:43 2022

@author: Lyle
"""

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

ALL_STRUCTURAL_DATASET = "/content/drive/MyDrive/all_structural_data_aug.csv"


def one_hot_encode_material(data):
    data = data.copy()
    # One-hot encode the materials
    data.loc[:, "Material"] = pd.Categorical(data["Material"], categories=["Steel", "Aluminum", "Titanium"])
    mats_oh = pd.get_dummies(data["Material"], prefix="Material=", prefix_sep="")
    data.drop(["Material"], axis=1, inplace=True)
    data = pd.concat([mats_oh, data], axis=1)
    return data


def load_augmented_framed_dataset():
    reg_data = pd.read_csv(ALL_STRUCTURAL_DATASET, index_col=0)

    x = reg_data.iloc[:, :-11]

    x = one_hot_encode_material(x)

    x, x_scaler = scale(x)
    y = reg_data.iloc[:, -11:-1]

    for col in ['Sim 1 Safety Factor', 'Sim 3 Safety Factor']:
        y[col] = 1 / y[col]
        y.rename(columns={col: col + " (Inverted)"}, inplace=True)
    for col in ['Sim 1 Dropout X Disp.', 'Sim 1 Dropout Y Disp.', 'Sim 1 Bottom Bracket X Disp.',
                'Sim 1 Bottom Bracket Y Disp.', 'Sim 2 Bottom Bracket Z Disp.', 'Sim 3 Bottom Bracket Y Disp.',
                'Sim 3 Bottom Bracket X Rot.', 'Model Mass']:
        y[col] = [np.abs(val) for val in y[col].values]
        y.rename(columns={col: col + " Magnitude"}, inplace=True)
    y, y_scaler = scale(y)

    return x, y, x_scaler, y_scaler


def scale(v):
    v_scaler = StandardScaler()
    v_scaler.fit(v)
    v_scaled_values = v_scaler.transform(v)
    new_v = pd.DataFrame(v_scaled_values, columns=v.columns, index=v.index)
    return new_v, v_scaler

In [5]:
x_scaled, y_scaled, x_scaler, y_scaler = load_augmented_framed_dataset()
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y_scaled, random_state=2023)


<ipython-input-4-8a703e428e7d>:18: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data.loc[:, "Material"] = pd.Categorical(data["Material"], categories=["Steel", "Aluminum", "Titanium"])


In [8]:
print(len(x_test), len(y_test))
len(x_train), len(y_train)

3713 3713


(11138, 11138)

In [9]:
full_training_set = pd.concat([x_train, y_train], axis=1)
len(full_training_set)

11138

In [10]:
my_predictor = MultilabelPredictor(labels=y_scaled.columns)
my_predictor.fit(
    train_data=full_training_set
)

Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230311_154304/Predictor_Sim 1 Dropout X Disp. Magnitude/"
AutoGluon Version:  0.7.0
Python Version:     3.9.16
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Dec 10 16:00:40 UTC 2022
Train Data Rows:    11138
Train Data Columns: 39
Label Column: Sim 1 Dropout X Disp. Magnitude
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (14.664168867583603, -0.8695634632320092, -0.00307, 0.9914)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11611

Fitting TabularPredictor for label: Sim 1 Dropout X Disp. Magnitude ...


	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 39 | ['Material=Steel', 'Material=Aluminum', 'Material=Titanium', 'SSB_Include', 'CSB_Include', ...]
	Types of features in processed data (raw dtype, special dtypes):
		('float', [])     : 34 | ['CS Length', 'BB Drop', 'Stack', 'SS E', 'ST Angle', ...]
		('int', ['bool']) :  5 | ['Material=Steel', 'Material=Aluminum', 'Material=Titanium', 'SSB_Include', 'CSB_Include']
	0.2s = Fit runtime
	39 features in original data used to generate 39 features in processed data.
	Train Data (Processed) Memory Usage: 3.09 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.2s ...
AutoGluon will gauge predictive performance using evaluation metric: 'root_mean_squared_error'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.
	To change this, specify the eval_metric parameter of Predictor()
Automaticall

[1000]	valid_set's rmse: 0.356845


	-0.355	 = Validation score   (-root_mean_squared_error)
	9.55s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: LightGBM ...
	-0.3369	 = Validation score   (-root_mean_squared_error)
	6.78s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-0.3869	 = Validation score   (-root_mean_squared_error)
	78.31s	 = Training   runtime
	0.16s	 = Validation runtime
Fitting model: CatBoost ...
	-0.35	 = Validation score   (-root_mean_squared_error)
	16.44s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-0.4165	 = Validation score   (-root_mean_squared_error)
	19.08s	 = Training   runtime
	0.29s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-0.3167	 = Validation score   (-root_mean_squared_error)
	16.5s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: XGBoost ...
	-0.3625	 = Validation score   (-root_mean_squared_error)
	5.86s	 = Training   runtime
	0.05s	 = Validation runtime
Fi

...

[1000]	valid_set's rmse: 0.139065
[2000]	valid_set's rmse: 0.137905
[3000]	valid_set's rmse: 0.137671
[4000]	valid_set's rmse: 0.137545
[5000]	valid_set's rmse: 0.137443
[6000]	valid_set's rmse: 0.137409
[7000]	valid_set's rmse: 0.137388
[8000]	valid_set's rmse: 0.137382
[9000]	valid_set's rmse: 0.137379
[10000]	valid_set's rmse: 0.137375


	-0.1374	 = Validation score   (-root_mean_squared_error)
	70.46s	 = Training   runtime
	1.45s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-0.2172	 = Validation score   (-root_mean_squared_error)
	90.23s	 = Training   runtime
	0.28s	 = Validation runtime
Fitting model: CatBoost ...
	-0.1095	 = Validation score   (-root_mean_squared_error)
	242.05s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-0.2049	 = Validation score   (-root_mean_squared_error)
	20.22s	 = Training   runtime
	0.16s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-0.1252	 = Validation score   (-root_mean_squared_error)
	13.05s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: XGBoost ...
	-0.1423	 = Validation score   (-root_mean_squared_error)
	32.65s	 = Training   runtime
	0.14s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	-0.1322	 = Validation score   (-root_mean_squared_error)
	43.27s	 = Training   runtime
	0.08s	 = Validat

[1000]	valid_set's rmse: 0.151324
[2000]	valid_set's rmse: 0.15108
[3000]	valid_set's rmse: 0.151059
[4000]	valid_set's rmse: 0.151057
[5000]	valid_set's rmse: 0.151056
[6000]	valid_set's rmse: 0.151056
[7000]	valid_set's rmse: 0.151056


	-0.1511	 = Validation score   (-root_mean_squared_error)
	185.56s	 = Training   runtime
	1.28s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	-0.1011	 = Validation score   (-root_mean_squared_error)
	0.4s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 764.32s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20230311_154304/Predictor_Model Mass Magnitude/")


MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('AutogluonModels/ag-20230311_154304/')


In [11]:
my_predictor = MultilabelPredictor.load("AutogluonModels/ag-20230311_154304/")
predictions = my_predictor.predict(x_test)
r2 = r2_score(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)

r2, mse, mae

(0.63394409450457, 0.21365510395757425, 0.12860334010343127)

In [12]:
import shutil

shutil.make_archive("trained-model-8pm", "zip", "/content/AutogluonModels")


'/content/trained-model-8pm.zip'